# Chat with Azure OpenAI vision models

**If you're looking for the web application, check the src/ folder.**

GPT-4o is the latest model from OpenAI. GPT-4o integrates text and images in a single model, enabling it to handle multiple data types simultaneously. This multimodal approach enhances accuracy and responsiveness in human-computer interactions. GPT-4o matches GPT-4 Turbo in English text and coding tasks while offering superior performance in non-English languages and vision tasks, setting new benchmarks for AI capabilities.

Documentation: https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-4o-and-gpt-4-turbo

In this lab we will try different variants of using GPT-4o model for image analysis

## Authenticate to OpenAI

The following code connects to OpenAI using an Azure OpenAI account. See the README for instruction on configuring the `.env` file.

In [ ]:
#import libraries
import os
from IPython.display import Image
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

os.getenv("AZURE_OPENAI_API_KEY")
    # Authenticate using an Azure OpenAI API key
    # This is generally discouraged, but is provided for developers
    # that want to develop locally inside the Docker container.
print("Using Azure OpenAI with key")
openai_client = openai.AzureOpenAI(
        api_version=os.getenv("AZURE_OPENAI_API_VERSION") or "2024-02-15-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    )

# Set the model name
OPENAI_MODEL= os.getenv("AZURE_OPENAI_MODEL")
print("Model name:", OPENAI_MODEL)

## 1. Drawing a Unicorn

In this exercise, you will use GPT-4 to generate step-by-step instructions on how to draw a unicorn using an image of aurochs as an input.  
The goal is to leverage the capabilities of the Azure OpenAI service to provide extensive analysis of images.

#### Send an image by web URL

In [ ]:
url =  "https://upload.wikimedia.org/wikipedia/commons/6/6e/Ur-painting.jpg" #image URL
prompt = "Is this a unicorn?" # text prompt to OpenAI model

# Send the prompt and image to the model
messages = [
    {
        "role": "user",
        "content": [
            {"text": prompt, "type": "text"},
            {
                "image_url": {"url": url},
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages, temperature=0.5)

# Display the image and the model's response
display(Image(url=url, width=200))
print(response.choices[0].message.content)

#### Send an image by Data URI



Create a function to read and encode image file for further processing

In [ ]:
# Helper function to open an image file and return it as a base64 encoded string
import base64

def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [ ]:
url = "../data/images/ur.jpg" #image file URL
prompt = "how could I make this into a unicorn though??" # text prompt to OpenAI model

# Send the prompt and image to the model
response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

# Display the image and the model's response
display(Image(url=url, width=200))
print(response.choices[0].message.content)

## 2. Graph analysis

In this exercise you will investigate GPT-4 ability to understand and and extract informationfrom graphs and charts.

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg/1280px-20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg.png"
prompt = "What zone are we losing the most trees in?"

messages = [
    {
        "role": "user",
        "content": [
            {"text": prompt, "type": "text"},
            {
                "image_url": {
                    "url": url
                },
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages, temperature=0.5)

display(Image(url=url, width=400))
print(response.choices[0].message.content)

## 3. Insurance claim processing

See how GPT-4 can help with business processes that include visual assessment, for example, insurance claim processing.

In [ ]:
url = "../data/images/dented_car.jpg"
response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "system",
            "content": (
                "You are an AI assistant that helps auto insurance companies process claims."
                "You accept images of damaged cars that are submitted with claims, and you are able to make judgments "
                "about the causes of automobile damage, and the validity of claims regarding that damage."
            ),
        },
        {
            "role": "user",
            "content": [
                {"text": "Claim states that this damage is due to hail. Is it valid?", "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        },
    ],
)

display(Image(url=url, width=400))
print(response.choices[0].message.content)

## 4. Appliance help

In [ ]:
url = "../data/images/dishwasher.png"
prompt = "How do I set this to wash the dishes quickly?"

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=800))
print(response.choices[0].message.content)

## 5. Assistance for vision-impaired

In [ ]:
url = "../data/images/menu.png"
prompt = "is there anything good for vegans on this menu?"

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=400))
print(response.choices[0].message.content)

## 6. Describe IT Architecture

In [ ]:
url = "../data/images/azure_arch.png"
prompt = "Suggest an alt text for this image"

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=400))
print(response.choices[0].message.content)

## 7. Table analysis

GPT-4o can understand and extract information from tables

In [ ]:
url = "../data/images/page_0.png"
prompt = "What's the cheapest plant?"

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=600))
print(response.choices[0].message.content)

## 8. Recommendation based on image analysis

GPT-4o can provide recommendations based on image content

In [ ]:
url = "../data/images/fridge.jpg"
prompt = "Propose me a meal based on the contents of my fridge"   

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=400))  
print(response.choices[0].message.content)

## 9. Challenge

Try your own images and different prompts for Image analysis with Azure OpenAI GPT-4o

1. Analyse image from file

In [ ]:
# Add your image path and prompt here 

url = "..." #add yor image path here
prompt = "..."   # add your prompt here

response = openai_client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": prompt, "type": "text"},
                {"image_url": {"url": open_image_as_base64(url)}, "type": "image_url"},
            ],
        }
    ],
)

display(Image(url=url, width=400))  
print(response.choices[0].message.content)

2. Analyse image from URL

In [ ]:
# Add your image url and prompt here

url =  "..." #add your image url here
prompt = "..."   # add your prompt here

messages = [
    {
        "role": "user",
        "content": [
            {"text": prompt, "type": "text"},
            {
                "image_url": {"url": url},
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages, temperature=0.5)

display(Image(url=url, width=200))
print(response.choices[0].message.content)